In [1]:
from __future__ import print_function
# 导入numpy库， numpy是一个常用的科学计算库，优化矩阵的运算
import numpy as np
import pandas as pd
import keras
# 导入顺序模型
from keras.models import Sequential
# 导入全连接层Dense， 激活层Activation 以及 Dropout层
from keras.layers.core import Dense, Dropout, Activation
# 导入优化器RMSProp
from keras.optimizers import RMSprop
# 导入numpy工具，主要是用to_categorical来转换类别向量
from keras.utils import np_utils
from scipy import sparse
import tensorflow as tf

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_x=sparse.load_npz('train_x_sparse.npz')
train_y=pd.read_csv("train_y.csv",header=None)
train_y=train_y[1]

column=pd.read_csv('./shuchu/index.csv')
index=list(column["0"])
index.remove('id')
index.remove('label')

train_x=pd.DataFrame(train_x.toarray(),columns=index)

test_x=sparse.load_npz('test_x_sparse.npz')
test_y=pd.read_csv("test_y.csv",header=None)
test_y=test_y[1]

test_x=pd.DataFrame(test_x.toarray(),columns=index)

In [3]:
mean=train_x.mean(skipna=True,axis=0)
std=train_x.std(skipna=True,axis=0)

train_x=(train_x-mean)/std
test_x=(test_x-mean)/std

In [4]:
train_x.fillna(-3,inplace=True)
test_x.fillna(-3,inplace=True)

#train_x.drop('tag',axis=1,inplace=True)
#test_x.drop('tag',axis=1,inplace=True)

In [5]:
train_x.head()

,loan_dt,tag,f135,f60,f36,f164,f8,f2,f119,f105,...,f6699,f6685,f6688,f6753,f6741,year,mon,mday,wday,queshi
0,0.784673,-0.668292,0.055600,-0.082115,0.408484,-0.010295,-0.011229,-0.005350,-0.011457,-0.009340,...,-0.594802,-0.705275,-0.254002,-1.218967,-1.138303,0.722689,-0.323464,-1.290513,0.586045,-1.189159
1,0.861303,-0.668292,-0.711186,0.376802,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,...,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,0.722689,-0.323464,-0.189227,1.633715,1.288711
2,0.818730,-0.668292,-0.729505,-0.082115,0.409014,-0.016913,-0.012936,0.001919,0.007911,-0.008932,...,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,0.722689,-0.323464,-0.801053,-0.985459,0.036734
3,-2.042122,-0.668292,-3.000000,-3.000000,0.427195,-0.017215,-0.013028,0.001618,0.007306,-0.009109,...,-0.594802,-0.705275,-0.254002,-1.218967,-1.138303,-1.383703,0.116169,-0.923418,-0.985459,-0.063250
4,0.767644,-0.668292,-0.598654,-0.082115,0.389742,-0.016840,-0.012851,0.001992,0.009878,-0.008769,...,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,0.722689,-0.323464,-1.535243,-0.461624,0.158454


In [6]:
np.sqrt(train_x.shape[1])

82.56512580987206

In [99]:

# 建立顺序型模型
model = Sequential()
'''
模型需要知道输入数据的shape，
因此，Sequential的第一层需要接受一个关于输入数据shape的参数，
后面的各个层则可以自动推导出中间数据的shape，
因此不需要为每个层都指定这个参数
''' 

# 输入层有6813个神经元
# 第一个隐层有83个神经元，激活函数为ReLu，Dropout比例为0.2

model.add(keras.layers.convolutional.Conv1D(2, 3,strides=3, padding='valid', input_shape=(train_x.shape[1],1),activation='relu'))

model.add(keras.layers.convolutional.Conv1D(2, 3,strides=3,padding='valid',activation='relu'))

model.add(keras.layers.convolutional.Conv1D(2, 3,strides=3, padding='valid',activation='relu'))

model.add(keras.layers.convolutional.Conv1D(2, 3,strides=3, padding='valid',activation='relu'))

model.add(keras.layers.core.Flatten(name="reshape_layer"))
model.add(Dense(30,activation='relu') )

model.add(Dense(30,activation='relu')) #没有input 表示隐层神经元
model.add(Dropout(0.3))

model.add(Dense(30,activation='relu')) #没有input 表示隐层神经元
model.add(Dropout(0.3))

model.add(Dense(30,activation='relu')) #没有input 表示隐层神经元
model.add(Dropout(0.3))



# 输出层有1个神经元，激活函数为sigmoid，得到分类结果
model.add(Dense(1))
model.add(Activation('sigmoid'))

# 输出模型的整体信息
# 总共参数数量为784*512+512 + 512*512+512 + 512*10+10 = 669706
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_42 (Conv1D)           (None, 2272, 2)           8         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 757, 2)            14        
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 252, 2)            14        
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 84, 2)             14        
_________________________________________________________________
reshape_layer (Flatten)      (None, 168)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 30)                5070      
_________________________________________________________________
dense_48 (Dense)             (None, 30)                930       
__________

In [17]:
model.layers

In [14]:
from keras import backend as K
# AUC for a binary classifier
def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)+0.01
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)+0.01
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P


In [100]:
'''
配置模型的学习过程
compile接收三个参数：
1.优化器optimizer：参数可指定为已预定义的优化器名，如rmsprop、adagrad，
或一个Optimizer类对象，如此处的RMSprop()
2.损失函数loss：参数为模型试图最小化的目标函数，可为预定义的损失函数，
如categorical_crossentropy、mse，也可以为一个损失函数
3.指标列表：对于分类问题，一般将该列表设置为metrics=['accuracy']
'''
from keras.optimizers import SGD, Adam
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[auc]) 

In [27]:
train_x= np.expand_dims(train_x, 2)
test_x= np.expand_dims(test_x, 2)



In [103]:
# 设置batch的大小
batch_size = 5000
# 设置迭代的次数
nb_epoch = 30

'''
训练模型
batch_size：指定梯度下降时每个batch包含的样本数
nb_epoch：训练的轮数，nb指number of
verbose：日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录，2为epoch输出一行记录
validation_data：指定验证集
fit函数返回一个History的对象，其History.history属性记录了损失函数和其他指标的数值随epoch变化的情况，
如果有验证集的话，也包含了验证集的这些指标变化情况
'''
history = model.fit(train_x, train_y,
                    batch_size = batch_size,
                    nb_epoch = nb_epoch,
                    verbose = 1,
                   validation_data = (test_x, test_y))

# 按batch计算在某些输入数据上模型的误差
score = model.evaluate(test_x, test_y, verbose=1)
# 输出训练好的模型在测试集上的表现
print('Test score:', score)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 75000 samples, validate on 25000 samples
Epoch 1/30
75000/75000 [==============================] - 48s 634us/step - loss: 0.2835 - auc: 0.7921 - val_loss: 0.2990 - val_auc: 0.7566
Epoch 2/30
75000/75000 [==============================] - 47s 620us/step - loss: 0.2827 - auc: 0.7935 - val_loss: 0.2993 - val_auc: 0.7560
Epoch 3/30
75000/75000 [==============================] - 47s 621us/step - loss: 0.2828 - auc: 0.7939 - val_loss: 0.3000 - val_auc: 0.7559
Epoch 4/30
75000/75000 [==============================] - 47s 632us/step - loss: 0.2824 - auc: 0.7936 - val_loss: 0.3011 - val_auc: 0.7577
Epoch 5/30
75000/75000 [==============================] - 47s 633us/step - loss: 0.2822 - auc: 0.7940 - val_loss: 0.3008 - val_auc: 0.7561
Epoch 6/30
75000/75000 [==============================] - 46s 617us/step - loss: 0.2804 - auc: 0.7978 - val_loss: 0.3006 - val_auc: 0.7566
Epoch 7/30
75000/75000 [==============================] - 47s 624us/step - loss: 0.2812 - auc: 0.7965 - val_loss: 0.

KeyboardInterrupt: 

In [12]:
score

[0.28847592805862426, 0.7894290420292318]

In [55]:
model.save('keras_all.h5')

In [80]:
import time, datetime
test_online=pd.read_table("valid.txt")
import time, datetime
def ymd(d):
    a=np.zeros(d.shape[0])
    b=np.zeros(d.shape[0])
    c=np.zeros(d.shape[0])
    e=np.zeros(d.shape[0])
    for i in range(d.shape[0]):
        timeArray = time.strptime(d.loan_dt[i], "%Y-%m-%d")
        a[i]=timeArray.tm_year
        b[i]=timeArray.tm_mon
        c[i]=timeArray.tm_mday
        e[i]=timeArray.tm_wday
    d['year']=a
    d['mon']=b
    d['mday']=c
    d['wday']=e
ymd(test_online)
def t(d):
    timeStamp=np.zeros(d.shape[0])
    for i in range(d.shape[0]):
        timeArray = time.strptime(d[i], "%Y-%m-%d")
        #print(timeArray)     
        # 转为时间戳
        timeStamp[i] = (int(time.mktime(timeArray))//86400)
        #print (timeStamp)  # 1381419600
    #print(timeStamp)
    return timeStamp-17256.0
test_online.loan_dt=t(test_online.loan_dt)
def queshi(d):
    缺失=np.isnan(d[:]).sum(axis=1)
    d['queshi']=round(((缺失-min(缺失))/(max(缺失)-min(缺失)))*10)
queshi(test_online)
test_online=test_online.drop(['id'],axis=1)

test_online['tag']=1

column=pd.read_csv('./shuchu/index.csv')

index=list(column["0"])
index.remove('id')
index.remove('label')
test_online=test_online[index]

test_online=(test_online-mean)/std
test_online.fillna(-3,inplace=True)

In [6]:
#test_online.drop('tag',axis=1,inplace=True)

In [81]:
test_x

array([[[ 0.74210045],
        [ 1.49633265],
        [-3.        ],
        ...,
        [-1.90233884],
        [ 1.63371466],
        [ 0.43232426]],

       [[ 0.89536037],
        [-0.66829169],
        [-3.        ],
        ...,
        [ 0.30023344],
        [ 0.06221061],
        [ 0.4801428 ]],

       [[ 0.71655713],
        [ 1.49633265],
        [-3.        ],
        ...,
        [ 1.52388471],
        [ 0.06221061],
        [ 1.63648215]],

       ...,

       [[ 0.66547049],
        [ 1.49633265],
        [-3.        ],
        ...,
        [ 0.78969395],
        [ 0.58604529],
        [ 0.61200606]],

       [[ 0.88684593],
        [-0.66829169],
        [-0.33695226],
        ...,
        [ 0.17786832],
        [-0.46162408],
        [-1.48766277]],

       [[ 0.98050478],
        [-0.66829169],
        [-3.        ],
        ...,
        [ 1.52388471],
        [ 1.63371466],
        [-0.33132341]]])

In [83]:
test_online.shape

(20000, 6817)

In [84]:
test_online= np.expand_dims(test_online, 2)

In [85]:
preds = model.predict(test_online)

In [92]:
max(preds)

array([0.6026944], dtype=float32)

In [89]:
v_id=pd.read_table("valid_id.txt")
f=pd.DataFrame(columns=["id","prob"])
f.prob=list(preds.T[0])
f.id=v_id
#print(f)
f.to_csv("result_keras_rawdata_tag.csv",encoding="utf-8")

(20000, 2)

# 迁移

In [5]:
train_x=sparse.load_npz('train_x_sparse.npz')
train_y=pd.read_csv("train_y.csv",header=None)
train_y=train_y[1]

column=pd.read_csv('./shuchu/index.csv')
index=list(column["0"])
index.remove('id')
index.remove('label')
train_x=pd.DataFrame(train_x.toarray(),columns=index)


test_x=sparse.load_npz('test_x_sparse.npz')
test_y=pd.read_csv("test_y.csv",header=None)
test_y=test_y[1]

test_x=pd.DataFrame(test_x.toarray(),columns=index)


mean=train_x.mean(skipna=True,axis=0)
std=train_x.std(skipna=True,axis=0)

train_x=(train_x-mean)/std

test_x=(test_x-mean)/std
train_x.fillna(-1,inplace=True)
test_x.fillna(-1,inplace=True)

In [9]:
data_tag=pd.read_csv('./shuchu/data_tag.csv')

In [10]:
from sklearn.cross_validation import train_test_split
train_tag,test_tag = train_test_split(data_tag.tag, test_size = 0.25,random_state=36)
train_x_tag_0=train_x.iloc[train_x.index[train_tag==0],:]
train_y_tag_0=train_y[train_y.index[train_tag==0]]
train_x_tag_1=train_x.iloc[train_x.index[train_tag==1],:]
train_y_tag_1=train_y[train_y.index[train_tag==1]]
test_x_tag_0=test_x.iloc[test_x.index[test_tag==0],:]
test_y_tag_0=test_y[test_y.index[test_tag==0]]
test_x_tag_1=test_x.iloc[test_x.index[test_tag==1],:]
test_y_tag_1=test_y[test_y.index[test_tag==1]]
del train_x
del test_x

In [24]:
#from keras.models import load_model  
#model = load_model('keras_all.h5',custom_objects={'auc':auc})

In [8]:
model.layers

In [ ]:
#for layer in model.layers[:3]:
#    layer.trainable = False

In [92]:
model = Sequential()
'''
模型需要知道输入数据的shape，
因此，Sequential的第一层需要接受一个关于输入数据shape的参数，
后面的各个层则可以自动推导出中间数据的shape，
因此不需要为每个层都指定这个参数
''' 

# 输入层有6813个神经元
# 第一个隐层有83个神经元，激活函数为ReLu，Dropout比例为0.2
model.add(Dense(600, input_shape=(train_x_tag_0.shape[1],),init='uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.2))


model.add(Dense(300,activation='relu',init='uniform')) #没有input 表示隐层神经元
model.add(Dropout(0.3))



model.add(Dense(83,activation='relu',init='uniform')) #没有input 表示隐层神经元
model.add(Dropout(0.3))


model.add(Dense(83,activation='relu',init='uniform')) #没有input 表示隐层神经元
model.add(Dropout(0.3))


model.add(Dense(83,activation='relu',init='uniform')) #没有input 表示隐层神经元
model.add(Dropout(0.3))

model.add(Dense(83,activation='relu',init='uniform')) #没有input 表示隐层神经元
model.add(Dropout(0.3))

# 输出层有1个神经元，激活函数为sigmoid，得到分类结果
model.add(Dense(1))
model.add(Activation('sigmoid'))

# 输出模型的整体信息
# 总共参数数量为784*512+512 + 512*512+512 + 512*10+10 = 669706
model.summary()

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(600, input_shape=(6812,), kernel_initializer="uniform")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(300, activation="relu", kernel_initializer="uniform")`
  app.launch_new_instance()
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(83, activation="relu", kernel_initializer="uniform")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_100 (Dense)            (None, 600)               4087800   
_________________________________________________________________
activation_29 (Activation)   (None, 600)               0         
_________________________________________________________________
dropout_80 (Dropout)         (None, 600)               0         
_________________________________________________________________
dense_101 (Dense)            (None, 300)               180300    
_________________________________________________________________
dropout_81 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_102 (Dense)            (None, 83)                24983     
_________________________________________________________________
dropout_82 (Dropout)         (None, 83)                0         
__________

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(83, activation="relu", kernel_initializer="uniform")`
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(83, activation="relu", kernel_initializer="uniform")`
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(83, activation="relu", kernel_initializer="uniform")`


In [93]:
np.random.seed(2018)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[auc]) 

In [94]:
# 设置batch的大小
batch_size = 5000
# 设置迭代的次数
nb_epoch = 8

'''
训练模型
batch_size：指定梯度下降时每个batch包含的样本数
nb_epoch：训练的轮数，nb指number of
verbose：日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录，2为epoch输出一行记录
validation_data：指定验证集
fit函数返回一个History的对象，其History.history属性记录了损失函数和其他指标的数值随epoch变化的情况，
如果有验证集的话，也包含了验证集的这些指标变化情况
'''
history = model.fit(train_x_tag_0, train_y_tag_0,
                    batch_size = batch_size,
                    nb_epoch = nb_epoch,
                    verbose = 1,
                    validation_data = (test_x_tag_0, test_y_tag_0))

# 按batch计算在某些输入数据上模型的误差
score = model.evaluate(test_x_tag_0, test_y_tag_0, verbose=1)
# 输出训练好的模型在测试集上的表现
print('Test score:', score)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 51845 samples, validate on 17320 samples
Epoch 1/8
51845/51845 [==============================] - 84s 2ms/step - loss: 0.5025 - auc: 0.5059 - val_loss: nan - val_auc: 0.6704
Epoch 2/8
51845/51845 [==============================] - 14s 267us/step - loss: 0.3774 - auc: 0.6677 - val_loss: nan - val_auc: 0.7278
Epoch 3/8
51845/51845 [==============================] - 13s 254us/step - loss: 0.3516 - auc: 0.7282 - val_loss: nan - val_auc: 0.7541
Epoch 4/8
51845/51845 [==============================] - 13s 257us/step - loss: 0.3361 - auc: 0.7586 - val_loss: nan - val_auc: 0.7711
Epoch 5/8
51845/51845 [==============================] - 13s 251us/step - loss: 0.3237 - auc: 0.7845 - val_loss: nan - val_auc: 0.7828
Epoch 6/8
51845/51845 [==============================] - 13s 249us/step - loss: 0.3133 - auc: 0.8069 - val_loss: nan - val_auc: 0.7944
Epoch 7/8
51845/51845 [==============================] - 14s 264us/step - loss: 0.3007 - auc: 0.8294 - val_loss: nan - val_auc: 0.8015
Epoch 8

In [32]:
model.save('keras_tag_0.h5')

In [50]:
preds = model.predict(test_online)
v_id=pd.read_table("valid_id.txt")
f=pd.DataFrame(columns=["id","prob"])
f.prob=list(preds.T[0])
f.id=v_id
#print(f)
f.to_csv("result_keras_rawdata_tag_0.csv",encoding="utf-8")

In [12]:
train_x_tag_0.shape

(51845, 6813)

In [78]:
from keras.models import load_model  
model = load_model('keras_tag_0.h5',custom_objects={'auc':auc})

In [53]:
model.layers

In [59]:
for layer in model.layers[:8]:
    layer.trainable = False
for layer in model.layers[8:]:
    layer.trainable = True
#model.layers[-1]=True
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 200)               1362600   
_________________________________________________________________
activation_9 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_29 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_30 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 100)               20100     
__________

/usr/local/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [ ]:
from keras.optimizers import SGD, Adam

adam=Adam(lr=0.1)
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=[auc]) 

In [60]:
# 设置batch的大小
batch_size = 5000
# 设置迭代的次数
nb_epoch = 10

'''
训练模型
batch_size：指定梯度下降时每个batch包含的样本数
nb_epoch：训练的轮数，nb指number of
verbose：日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录，2为epoch输出一行记录
validation_data：指定验证集
fit函数返回一个History的对象，其History.history属性记录了损失函数和其他指标的数值随epoch变化的情况，
如果有验证集的话，也包含了验证集的这些指标变化情况
'''

history = model.fit(train_x_tag_1, train_y_tag_1,
                    batch_size = batch_size,
                    nb_epoch = nb_epoch,
                    verbose = 1,
                    validation_data = (test_x_tag_1, test_y_tag_1))

# 按batch计算在某些输入数据上模型的误差
score = model.evaluate(test_x_tag_1, test_y_tag_1, verbose=1)
# 输出训练好的模型在测试集上的表现
print('Test score:', score)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 23155 samples, validate on 7680 samples
Epoch 1/10
20000/23155 [========================>.....] - ETA: 6s - loss: 0.9252 - auc: 2.0861e-06 

KeyboardInterrupt: 

In [27]:
score = model.evaluate(test_x_tag_0, test_y_tag_0, verbose=1)
# 输出训练好的模型在测试集上的表现
print('Test score:', score)

17320/17320 [==============================] - 16s 902us/step
Test score: [nan, 0.689121961566097]


In [40]:
model.save('keras_tag_1.h5')

In [84]:

preds = model.predict(test_online)
v_id=pd.read_table("valid_id.txt")
f=pd.DataFrame(columns=["id","prob"])
f.prob=list(preds.T[0])
f.id=v_id
#print(f)

In [48]:
f.to_csv("result_keras_rawdata_tag_1.csv",encoding="utf-8")

In [71]:
w=pd.read_csv("./shuchu/re_tag.csv")
#(w.prob>0.47992188)
w.drop('Unnamed: 0',axis=1,inplace=True)
sum(w.prob<0.47992188)

1242

In [66]:
r1=pd.read_csv("result_keras_rawdata_tag_1.csv")
r1.drop('Unnamed: 0',axis=1,inplace=True)
r0=pd.read_csv("result_keras_rawdata_tag_0.csv")
print(r1.head())

       id      prob
0  100001  0.163625
1  100002  0.101252
2  100003  0.076871
3  100004  0.004411
4  100005  0.003073


In [67]:
r1.iloc[r1.index[w.prob<0.47992188],1]=r0.prob[r1.index[w.prob<0.47992188]]

In [68]:
r1.head()

,id,prob
0,100001,0.163625
1,100002,0.101252
2,100003,0.076871
3,100004,0.008801
4,100005,0.003073


In [70]:
r1.to_csv("result_keras.csv",encoding="utf-8")